In [1]:
import requests
import pandas as pd
from functools import *
import numpy as np
import nltk
%load_ext autoreload
%autoreload 2
import recip as rr
import dill

In [2]:
inp = ["flour","butter","milk", "egg", "cheese"]

In [5]:
# allr = rr.GetRecipies(inp)
# allr.get()

In [3]:
allr = rr.GetRecipies(inp)
dbfile = open('resp.pkl', 'rb')      
allr.response =  dill.load(dbfile)

In [4]:
allr.process()

In [17]:
allr.recser.iloc[0].id

14438